In [ ]:
from typing import List

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from tensorflow import keras
from keras import backend
from keras import layers

tfds.disable_progress_bar()
keras.utils.set_random_seed(42)

In [ ]:
# Model
IMAGE_SIZE = 224
PATCH_SIZE = 16
NUM_PATCHES = (IMAGE_SIZE // PATCH_SIZE) ** 2
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 192
NUM_HEADS = 3
NUM_LAYERS = 12
MLP_UNITS = [
    PROJECTION_DIM * 4,
    PROJECTION_DIM,
]
DROPOUT_RATE = 0.0

# Training
NUM_EPOCHS = 10
BASE_LR = 0.0005
WEIGHT_DECAY = 0.0001

# Data
BATCH_SIZE = 64
AUTO = tf.data.AUTOTUNE
NUM_CLASSES = 5

In [ ]:
def preprocess_dataset(is_training=True):
    def fn(image, label):
        if is_training:
            # Resize to a bigger spatial resolution and take the random crops.
            image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
            image = tf.image.random_crop(image, (IMAGE_SIZE, IMAGE_SIZE, 3))
            image = tf.image.random_flip_left_right(image)
            image = image / 255.0
        else:
            image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
            image = image / 255.0
        label = tf.one_hot(label, depth=NUM_CLASSES)
        return image, label

    return fn


def prepare_dataset(dataset, is_training=True):
    if is_training:
        dataset = dataset.shuffle(BATCH_SIZE * 10)
    dataset = dataset.map(preprocess_dataset(is_training), num_parallel_calls=AUTO)
    return dataset.batch(BATCH_SIZE).prefetch(AUTO)


train_dataset, val_dataset = tfds.load(
    "tf_flowers", split=["train[:90%]", "train[90%:]",], as_supervised=True
)
num_train = train_dataset.cardinality()
num_val = val_dataset.cardinality()

print(f"Number of training examples: {num_train}")
print(f"Number of validation examples: {num_val}")


train_dataset = prepare_dataset(train_dataset, is_training=True)
val_dataset = prepare_dataset(val_dataset, is_training=False)


Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.
Number of training examples: 3303
Number of validation examples: 367


In [ ]:
def mlp(x, dropout_rate: float, hidden_units: List):
    """FFN for a Transformer block."""
    # Iterate over the hidden units and add Dropout.
    for (idx, units) in enumerate(hidden_units):
        x = layers.Dense(
            units,
            activation=tf.nn.gelu if idx == 0 else None,
        )(x)
        x = layers.Dropout(dropout_rate)(x)
    return x


def transformer(name: str) -> keras.Model:
    """Transformer block with pre-norm."""
    num_patches = NUM_PATCHES + 2 if "deit" in MODEL_TYPE else NUM_PATCHES + 1
    encoded_patches = layers.Input((num_patches, PROJECTION_DIM))

    # Implement here
    # Layer normalization 1 - layers.LayerNormalization(epsilon)

    # Multi Head Self Attention layer - layers.MultiHeadAttention(num_heads, key_dim, dropout)

    # Skip connection
    # layers.add()

    # Layer normalization 2

    # MLP layer 1.

    # Skip connection 2.

    return keras.Model(encoded_patches, outputs, name=name)


In [ ]:
class ViTClassifier(keras.Model):
    """Vision Transformer base class."""
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        # Patchify + linear projection + reshaping.
        self.projection = keras.Sequential(
            [
                layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
                # Implement here
                # Hint: Use convolution

                #layers.Conv2D()

                #layers.Reshape()
            ],
            name="projection",
        )

        # Positional embedding.
        init_shape = (
            1,
            NUM_PATCHES + 1,
            PROJECTION_DIM,
        )
        self.positional_embedding = tf.Variable(
            tf.zeros(init_shape), name="pos_embedding"
        )

        # Transformer blocks.
        self.transformer_blocks = [
            transformer(name=f"transformer_block_{i}")
            for i in range(NUM_LAYERS)
        ]

        # CLS token.
        initial_value = tf.zeros((1, 1, PROJECTION_DIM))
        self.cls_token = tf.Variable(
            initial_value=initial_value, trainable=True, name="cls"
        )

        # Other layers.
        self.dropout = layers.Dropout(DROPOUT_RATE)
        self.layer_norm = layers.LayerNormalization(epsilon=LAYER_NORM_EPS)
        self.head = layers.Dense(
            NUM_CLASSES,
            name="classification_head",
        )

    def call(self, inputs, training=True):
        n = tf.shape(inputs)[0]

        # Create patches and project the patches.
        projected_patches = self.projection(inputs)

        # Append class token if needed.
        cls_token = tf.tile(self.cls_token, (n, 1, 1))
        cls_token = tf.cast(cls_token, projected_patches.dtype)
        projected_patches = tf.concat([cls_token, projected_patches], axis=1)

        # Add positional embeddings to the projected patches.
        encoded_patches = (
            self.positional_embedding + projected_patches
        )
        encoded_patches = self.dropout(encoded_patches)

        # Iterate over the number of layers and stack up blocks of Transformer.
        for transformer_module in self.transformer_blocks:
            # Add a Transformer block.
            encoded_patches = transformer_module(encoded_patches)

        # Final layer normalization.
        representation = self.layer_norm(encoded_patches)

        # Pool representation.
        encoded_patches = representation[:, 0]

        # Classification head.
        output = self.head(encoded_patches)
        return output


In [ ]:
MODEL_TYPE = "vit"
vit_model = ViTClassifier()

vit_model.compile(
    optimizer=keras.optimizers.AdamW(learning_rate=BASE_LR/51200 * BATCH_SIZE, weight_decay=WEIGHT_DECAY),
    metrics=["accuracy"],
    loss='categorical_crossentropy',
)

_ = vit_model.fit(train_dataset, validation_data=val_dataset, epochs=NUM_EPOCHS)

Epoch 1/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 102s 790ms/step - accuracy: 0.1618 - loss: 8.0384 - val_accuracy: 0.1526 - val_loss: 9.4425
Epoch 2/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 20s 380ms/step - accuracy: 0.1471 - loss: 8.1811 - val_accuracy: 0.1526 - val_loss: 9.2668
Epoch 3/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 21s 395ms/step - accuracy: 0.1386 - loss: 8.3209 - val_accuracy: 0.1526 - val_loss: 9.3986
Epoch 4/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 41s 404ms/step - accuracy: 0.1421 - loss: 8.2413 - val_accuracy: 0.1526 - val_loss: 9.3986
Epoch 5/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 41s 408ms/step - accuracy: 0.1425 - loss: 8.2497 - val_accuracy: 0.1526 - val_loss: 9.3986
Epoch 6/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 21s 408ms/step - accuracy: 0.1383 - loss: 8.0517 - val_accuracy: 0.1526 - val_loss: 9.3986
Epoch 7/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 40s 395ms/step - accuracy: 0.1458 - loss: 8.2998 - val_accuracy: 0.1526 - val_loss: 9.3986
Epoch 8/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 42s 411ms/step - accuracy: 0.1451 - loss: 8.2397 - val_acc

In [ ]:
_, vit_acc = vit_model.evaluate(val_dataset)

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - accuracy: 0.1570 - loss: 9.4360


# MobileViT: A mobile-friendly Transformer-based model for image classification

**Author:** [Sayak Paul](https://twitter.com/RisingSayak)<br>

**Description:** MobileViT for image classification with combined benefits of convolutions and Transformers.

In [ ]:
# Values are from table 4.
patch_size = 4  # 2x2, for the Transformer blocks.
image_size = 256
expansion_factor = 2  # expansion factor for the MobileNetV2 blocks.

In [ ]:
batch_size = 64
auto = tf.data.AUTOTUNE
resize_bigger = 280
num_classes = 5


def preprocess_dataset(is_training=True):
    def _pp(image, label):
        if is_training:
            # Resize to a bigger spatial resolution and take the random
            # crops.
            image = tf.image.resize(image, (resize_bigger, resize_bigger))
            image = tf.image.random_crop(image, (image_size, image_size, 3))
            image = tf.image.random_flip_left_right(image)
        else:
            image = tf.image.resize(image, (image_size, image_size))
        label = tf.one_hot(label, depth=num_classes)
        return image, label

    return _pp


def prepare_dataset(dataset, is_training=True):
    if is_training:
        dataset = dataset.shuffle(batch_size * 10)
    dataset = dataset.map(preprocess_dataset(is_training), num_parallel_calls=auto)
    return dataset.batch(batch_size).prefetch(auto)

train_dataset, val_dataset = tfds.load(
    "tf_flowers", split=["train[:90%]", "train[90%:]"], as_supervised=True
)

num_train = train_dataset.cardinality()
num_val = val_dataset.cardinality()
print(f"Number of training examples: {num_train}")
print(f"Number of validation examples: {num_val}")

train_dataset = prepare_dataset(train_dataset, is_training=True)
val_dataset = prepare_dataset(val_dataset, is_training=False)

Number of training examples: 3303
Number of validation examples: 367


## MobileViT utilities

The MobileViT architecture is comprised of the following blocks:

* Strided 3x3 convolutions that process the input image.
* [MobileNetV2](https://arxiv.org/abs/1801.04381)-style inverted residual blocks for
downsampling the resolution of the intermediate feature maps.
* MobileViT blocks that combine the benefits of Transformers and convolutions. It is
presented in the figure below (taken from the
[original paper](https://arxiv.org/abs/2110.02178)):


![](https://i.imgur.com/mANnhI7.png)

In [ ]:
def conv_block(x, filters=16, kernel_size=3, strides=2):
    conv_layer = layers.Conv2D(
        filters,
        kernel_size,
        strides=strides,
        activation=keras.activations.swish,
        padding="same",
    )
    return conv_layer(x)


# Reference: https://github.com/keras-team/keras/blob/e3858739d178fe16a0c77ce7fab88b0be6dbbdc7/keras/applications/imagenet_utils.py#L413C17-L435


def correct_pad(inputs, kernel_size):
    img_dim = 2 if backend.image_data_format() == "channels_first" else 1
    input_size = inputs.shape[img_dim : (img_dim + 2)]
    if isinstance(kernel_size, int):
        kernel_size = (kernel_size, kernel_size)
    if input_size[0] is None:
        adjust = (1, 1)
    else:
        adjust = (1 - input_size[0] % 2, 1 - input_size[1] % 2)
    correct = (kernel_size[0] // 2, kernel_size[1] // 2)
    return (
        (correct[0] - adjust[0], correct[0]),
        (correct[1] - adjust[1], correct[1]),
    )


# Reference: https://git.io/JKgtC


def inverted_residual_block(x, expanded_channels, output_channels, strides=1):
    # 1x1 Conv
    m = layers.Conv2D(expanded_channels, 1, padding="same", use_bias=False)(x)
    m = layers.BatchNormalization()(m)
    m = keras.activations.swish(m)

    # 3x3 DepthwiseConv2D
    if strides == 2:
        m = layers.ZeroPadding2D(padding=correct_pad(m, 3))(m)
    m = layers.DepthwiseConv2D(
        3, strides=strides, padding="same" if strides == 1 else "valid", use_bias=False
    )(m)
    m = layers.BatchNormalization()(m)
    m = keras.activations.swish(m)

    # 1x1 Conv
    m = layers.Conv2D(output_channels, 1, padding="same", use_bias=False)(m)
    m = layers.BatchNormalization()(m)

    # Skip Connection
    if keras.ops.equal(x.shape[-1], output_channels) and strides == 1:
        return layers.Add()([m, x])
    return m


# Reference:
# https://keras.io/examples/vision/image_classification_with_vision_transformer/


def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=keras.activations.swish)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x


def transformer_block(x, transformer_layers, projection_dim, num_heads=2):
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(x)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, x])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(
            x3,
            hidden_units=[x.shape[-1] * 2, x.shape[-1]],
            dropout_rate=0.1,
        )
        # Skip connection 2.
        x = layers.Add()([x3, x2])

    return x


def mobilevit_block(x, num_blocks, projection_dim, strides=1):
    # Local projection with convolutions.
    local_features = conv_block(x, filters=projection_dim, strides=strides)
    local_features = conv_block(
        local_features, filters=projection_dim, kernel_size=1, strides=strides
    )

    # Unfold into patches and then pass through Transformers.
    num_patches = int((local_features.shape[1] * local_features.shape[2]) / patch_size)
    non_overlapping_patches = layers.Reshape((patch_size, num_patches, projection_dim))(
        local_features
    )
    global_features = transformer_block(
        non_overlapping_patches, num_blocks, projection_dim
    )

    # Fold into conv-like feature-maps.
    folded_feature_map = layers.Reshape((*local_features.shape[1:-1], projection_dim))(
        global_features
    )

    # Apply point-wise conv -> concatenate with the input features.
    folded_feature_map = conv_block(
        folded_feature_map, filters=x.shape[-1], kernel_size=1, strides=strides
    )
    local_global_features = layers.Concatenate(axis=-1)([x, folded_feature_map])

    # Fuse the local and global features using a convoluion layer.
    local_global_features = conv_block(
        local_global_features, filters=projection_dim, strides=strides
    )

    return local_global_features


In [ ]:
def create_mobilevit(num_classes=5):
    inputs = keras.Input((image_size, image_size, 3))
    x = layers.Rescaling(scale=1.0 / 255)(inputs)

    # Initial conv-stem -> MV2 block.


    # Downsampling with MV2 block.


    # First MV2 -> MobileViT block.


    # Second MV2 -> MobileViT block.


    # Third MV2 -> MobileViT block.


    # Classification head.

    return keras.Model(inputs, outputs)


mobilevit_xxs = create_mobilevit()

In [ ]:
mobilevit_xxs.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.002),
    metrics=["accuracy"],
    loss='categorical_crossentropy',
)

_ = mobilevit_xxs.fit(train_dataset, validation_data=val_dataset, epochs=NUM_EPOCHS)

Epoch 1/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 203s 2s/step - accuracy: 0.4243 - loss: 1.3499 - val_accuracy: 0.2316 - val_loss: 1.9340
Epoch 2/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 61s 421ms/step - accuracy: 0.5983 - loss: 1.0022 - val_accuracy: 0.1907 - val_loss: 1.8497
Epoch 3/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 41s 419ms/step - accuracy: 0.6648 - loss: 0.8749 - val_accuracy: 0.1907 - val_loss: 1.9939
Epoch 4/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 41s 423ms/step - accuracy: 0.7079 - loss: 0.7831 - val_accuracy: 0.1907 - val_loss: 1.7146
Epoch 5/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 41s 423ms/step - accuracy: 0.7211 - loss: 0.7530 - val_accuracy: 0.1907 - val_loss: 2.4908
Epoch 6/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 41s 423ms/step - accuracy: 0.7297 - loss: 0.6925 - val_accuracy: 0.1907 - val_loss: 4.1751
Epoch 7/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 41s 424ms/step - accuracy: 0.7484 - loss: 0.6565 - val_accuracy: 0.1907 - val_loss: 2.4273
Epoch 8/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 40s 409ms/step - accuracy: 0.7736 - loss: 0.6251 - val_accura

In [ ]:
_, mobvit_acc = mobilevit_xxs.evaluate(val_dataset)

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - accuracy: 0.4640 - loss: 1.2604


## DeiT

In [ ]:
# Model
IMAGE_SIZE = 224
PATCH_SIZE = 16
NUM_PATCHES = (IMAGE_SIZE // PATCH_SIZE) ** 2
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 192
NUM_HEADS = 3
NUM_LAYERS = 12
MLP_UNITS = [
    PROJECTION_DIM * 4,
    PROJECTION_DIM,
]
DROPOUT_RATE = 0.0

# Training
NUM_EPOCHS = 10
BASE_LR = 0.0005
WEIGHT_DECAY = 0.0001

# Data
BATCH_SIZE = 64
AUTO = tf.data.AUTOTUNE
NUM_CLASSES = 5

In [ ]:
def preprocess_dataset(is_training=True):
    def fn(image, label):
        if is_training:
            # Resize to a bigger spatial resolution and take the random crops.
            image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
            image = tf.image.random_crop(image, (IMAGE_SIZE, IMAGE_SIZE, 3))
            image = tf.image.random_flip_left_right(image)
            image = image / 255.0
        else:
            image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
            image = image / 255.0
        label = tf.one_hot(label, depth=NUM_CLASSES)
        return image, label

    return fn


def prepare_dataset(dataset, is_training=True):
    if is_training:
        dataset = dataset.shuffle(BATCH_SIZE * 10)
    dataset = dataset.map(preprocess_dataset(is_training), num_parallel_calls=AUTO)
    return dataset.batch(BATCH_SIZE).prefetch(AUTO)


train_dataset, val_dataset = tfds.load(
    "tf_flowers", split=["train[:90%]", "train[90%:]",], as_supervised=True
)
num_train = train_dataset.cardinality()
num_val = val_dataset.cardinality()

print(f"Number of training examples: {num_train}")
print(f"Number of validation examples: {num_val}")


train_dataset = prepare_dataset(train_dataset, is_training=True)
val_dataset = prepare_dataset(val_dataset, is_training=False)


Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.
Number of training examples: 3303
Number of validation examples: 367


In [ ]:
class ViTDistilled(ViTClassifier):
    def __init__(self, regular_training=False, **kwargs):
        super().__init__(**kwargs)
        self.num_tokens = 2
        self.regular_training = regular_training

        # CLS and distillation tokens, positional embedding.
        init_value = tf.zeros((1, 1, PROJECTION_DIM))
        self.dist_token = tf.Variable(init_value, name="dist_token")
        self.positional_embedding = tf.Variable(
            tf.zeros(
                (
                    1,
                    NUM_PATCHES + self.num_tokens,
                    PROJECTION_DIM,
                )
            ),
            name="pos_embedding",
        )

        # Head layers.
        self.head = layers.Dense(
            NUM_CLASSES,
            name="classification_head",
        )
        self.head_dist = layers.Dense(
            NUM_CLASSES,
            name="distillation_head",
        )

    def call(self, inputs, training=True):
        n = tf.shape(inputs)[0]

        # Create patches and project the patches.
        projected_patches = self.projection(inputs)

        # Append the tokens.
        cls_token = tf.tile(self.cls_token, (n, 1, 1))
        dist_token = tf.tile(self.dist_token, (n, 1, 1))
        cls_token = tf.cast(cls_token, projected_patches.dtype)
        dist_token = tf.cast(dist_token, projected_patches.dtype)
        projected_patches = tf.concat(
            [cls_token, dist_token, projected_patches], axis=1
        )

        # Add positional embeddings to the projected patches.
        encoded_patches = (
            self.positional_embedding + projected_patches
        )
        encoded_patches = self.dropout(encoded_patches)

        # Iterate over the number of layers and stack up blocks of Transformer.
        for transformer_module in self.transformer_blocks:
            # Add a Transformer block.
            encoded_patches = transformer_module(encoded_patches)

        # Final layer normalization.
        representation = self.layer_norm(encoded_patches)

        # Classification heads.
        x, x_dist = (
            self.head(representation[:, 0]),
            self.head_dist(representation[:, 1]),
        )

        if not training or self.regular_training:
            # During standard train / finetune, inference average the classifier predictions.
            return (x + x_dist) / 2

        elif training:
            # Only return separate classification predictions when training in distilled mode.
            return x, x_dist


In [ ]:
class DeiT(keras.Model):
    # Reference:
    # https://keras.io/examples/vision/knowledge_distillation/
    def __init__(self, student, teacher, **kwargs):
        super().__init__(**kwargs)
        self.student = student
        self.teacher = teacher

        self.student_loss_tracker = keras.metrics.Mean(name="student_loss")
        self.dist_loss_tracker = keras.metrics.Mean(name="distillation_loss")

    @property
    def metrics(self):
        metrics = super().metrics
        metrics.append(self.student_loss_tracker)
        metrics.append(self.dist_loss_tracker)
        return metrics

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
    ):
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn

    def train_step(self, data):
        # Unpack data.
        x, y = data

        # Forward pass of teacher
        teacher_output = self.teacher(x)

        if isinstance(teacher_output, dict):
          logits = teacher_output['dense']
        else:
          logits = teacher_output

        teacher_probs = tf.nn.softmax(logits, -1)

        with tf.GradientTape() as tape:
            # Forward pass of student.
            cls_predictions, dist_predictions = self.student(x, training=True)

            # Compute losses.
            student_loss = self.student_loss_fn(y, cls_predictions)
            distillation_loss = self.distillation_loss_fn(
                teacher_probs, dist_predictions
            )
            loss = (student_loss + distillation_loss) / 2

        # Compute gradients.
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights.
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        student_predictions = (cls_predictions + dist_predictions) / 2
        self.compiled_metrics.update_state(y, student_predictions)
        self.dist_loss_tracker.update_state(distillation_loss)
        self.student_loss_tracker.update_state(student_loss)

        # Return a dict of performance.
        results = {m.name: m.result() for m in self.metrics}
        return results

    def test_step(self, data):
        # Unpack the data.
        x, y = data

        # Compute predictions.
        y_prediction = self.student(x, training=False)

        # Calculate the loss.
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)
        self.student_loss_tracker.update_state(student_loss)

        # Return a dict of performance.
        results = {m.name: m.result() for m in self.metrics}
        return results

    def call(self, inputs):
        return self.student(inputs, training=False)


In [ ]:
!wget -q https://github.com/sayakpaul/deit-tf/releases/download/v0.1.0/bit_teacher_flowers.zip
!unzip -q bit_teacher_flowers.zip


In [ ]:
MODEL_TYPE = "deit"
# bit_teacher_flowers = keras.models.load_model("bit_teacher_flowers")
bit_teacher_flowers = keras.layers.TFSMLayer("bit_teacher_flowers", call_endpoint="serving_default")

deit_tiny = ViTDistilled()
deit_distiller = DeiT(student=deit_tiny, teacher=bit_teacher_flowers)

lr_scaled = (0.1 / 512) * BATCH_SIZE
deit_distiller.compile(
    optimizer=keras.optimizers.AdamW(weight_decay=WEIGHT_DECAY, learning_rate=lr_scaled),
    metrics=["accuracy"],
    student_loss_fn=keras.losses.CategoricalCrossentropy(
        from_logits=True, label_smoothing=0.1
    ),
    distillation_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
)
_ = deit_distiller.fit(train_dataset, validation_data=val_dataset, epochs=NUM_EPOCHS)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'vi_t_distilled_5', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:642: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


52/52 ━━━━━━━━━━━━━━━━━━━━ 167s 2s/step - accuracy: 0.2173 - distillation_loss: 0.2560 - loss: 0.2420 - student_loss: 0.2620 - val_distillation_loss: 0.5151 - val_loss: 0.5151 - val_student_loss: 0.5256
Epoch 2/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 95s 2s/step - accuracy: 0.2177 - distillation_loss: 0.5080 - loss: 0.5007 - student_loss: 0.5097 - val_distillation_loss: 0.4887 - val_loss: 0.4887 - val_student_loss: 0.4987
Epoch 3/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 0.2386 - distillation_loss: 0.4874 - loss: 0.4800 - student_loss: 0.4892 - val_distillation_loss: 0.4641 - val_loss: 0.4641 - val_student_loss: 0.4750
Epoch 4/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.2320 - distillation_loss: 0.4594 - loss: 0.4518 - student_loss: 0.4612 - val_distillation_loss: 0.4359 - val_loss: 0.4359 - val_student_loss: 0.4462
Epoch 5/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 84s 2s/step - accuracy: 0.2455 - distillation_loss: 0.4235 - loss: 0.4156 - student_loss: 0.4250 - val_distillation_los

In [ ]:
_ = deit_distiller.evaluate(val_dataset)

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 141ms/step - accuracy: 0.3070 - distillation_loss: 0.1785 - loss: 0.1785 - student_loss: 0.1901
